In [1]:
import csv
import os
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

# Style for plots
sns.set(style="whitegrid", palette="pastel", font_scale=1.2)

# CSV setup
filename = "finance_log.csv"
fields = ["Date", "Day", "Transport", "Food", "Data", "School", "Clothing", "Others", "Savings", "Loans"]

if not os.path.exists(filename):
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(fields)


In [2]:
def save_entry(entry):
    with open(filename, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(entry)

def load_data():
    if not os.path.exists(filename):
        return None
    df = pd.read_csv(filename)
    if df.empty:
        return None
    # Normalize columns
    df.columns = df.columns.str.strip()
    # Ensure all expected fields exist
    for col in fields:
        if col not in df.columns:
            if col in fields[2:]:
                df[col] = 0
            else:
                df[col] = ""
    df[fields[2:]] = df[fields[2:]].apply(pd.to_numeric, errors='coerce').fillna(0)
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    return df


In [3]:
def input_daily():
    while True:
        day = input("Enter day (Monday-Saturday) or 'q' to quit: ")
        if day.lower() == 'q':
            break
        date_input = input("Enter date (DD-MM-YYYY) or leave blank for today: ")
        if not date_input:
            date_input = datetime.today().strftime("%d-%m-%Y")
        try:
            datetime.strptime(date_input, "%d-%m-%Y")
        except:
            print("Invalid date format.")
            continue
        amounts = input("Enter Transport, Food, Data, School, Clothing, Others, Savings, Loans (comma-separated): ")
        try:
            values = [float(x.strip()) for x in amounts.split(",")]
            if len(values) != 8:
                print("Enter exactly 8 numbers.")
                continue
            save_entry([date_input, day] + values)
        except:
            print("Invalid input, make sure all are numbers.")
            continue
        cont = input("Continue for next day? (yes/no): ")
        if cont.lower() != 'yes':
            break

def input_weekly():
    while True:
        date_input = input("Enter start date of week (DD-MM-YYYY) or leave blank for today: ")
        if not date_input:
            date_input = datetime.today().strftime("%d-%m-%Y")
        try:
            week_num = datetime.strptime(date_input, "%d-%m-%Y").isocalendar()[1]
        except:
            print("Invalid date format.")
            continue
        amounts = input("Enter Transport, Food, Data, School, Clothing, Others, Savings, Loans (comma-separated): ")
        try:
            values = [float(x.strip()) for x in amounts.split(",")]
            if len(values) != 8:
                print("Enter exactly 8 numbers.")
                continue
            save_entry([date_input, f"Week {week_num}"] + values)
        except:
            print("Invalid input.")
            continue
        cont = input(f"Continue for week {week_num}? (yes/no): ")
        if cont.lower() != 'yes':
            break

def input_monthly():
    while True:
        date_input = input("Enter date (DD-MM-YYYY) or leave blank for today: ")
        if not date_input:
            date_input = datetime.today().strftime("%d-%m-%Y")
        amounts = input("Enter Transport, Food, Data, School, Clothing, Others, Savings, Loans (comma-separated): ")
        try:
            values = [float(x.strip()) for x in amounts.split(",")]
            if len(values) != 8:
                print("Enter exactly 8 numbers.")
                continue
            save_entry([date_input, "Month"] + values)
        except:
            print("Invalid input.")
            continue
        cont = input("Continue entering for next month? (yes/no): ")
        if cont.lower() != 'yes':
            break


In [4]:
def view_spending(period="daily"):
    df = load_data()
    if df is None:
        print("No data available.")
        return
    if period == "daily":
        print(df.groupby("Date")[fields[2:]].sum())
    elif period == "weekly":
        df['Week'] = df['Date'].dt.isocalendar().week
        print(df.groupby("Week")[fields[2:]].sum())
    elif period == "monthly":
        df['Month'] = df['Date'].dt.month
        print(df.groupby("Month")[fields[2:]].sum())


In [ ]:
df = load_data()
if df is None:
    print("No data available.")
else:
    df.columns = df.columns.str.strip()
    available_categories = [c for c in fields[2:] if c in df.columns]
    
    cat_select = widgets.SelectMultiple(options=available_categories, value=[available_categories[0]], description='Categories:')
    chart_type = widgets.Dropdown(options=['Bar','Line','Pie','Stacked Bar','Cumulative Line'], value='Bar', description='Chart Type:')
    start_default = df['Date'].min().date()
    end_default = df['Date'].max().date()
    date_start = widgets.DatePicker(description='Start Date', value=start_default)
    date_end = widgets.DatePicker(description='End Date', value=end_default)
    btn = widgets.Button(description="Visualize")
    output = widgets.Output()

    def plot_spending(change):
        with output:
            clear_output()
            selected_cats = [c for c in list(cat_select.value) if c in df.columns]
            if not selected_cats:
                print("No valid categories selected.")
                return
            try:
                start = pd.to_datetime(date_start.value) if date_start.value else df['Date'].min()
                end = pd.to_datetime(date_end.value) if date_end.value else df['Date'].max()
            except:
                print("Invalid dates")
                return
            if start > end:
                print("Start date must be before end date")
                return
            mask = (df['Date'] >= start) & (df['Date'] <= end)
            data = df.loc[mask, ['Date'] + selected_cats].copy()
            if data.empty:
                print("No data in selected range.")
                return
            data.set_index('Date', inplace=True)
            plt.figure(figsize=(12,6))
            try:
                if chart_type.value == 'Bar':
                    data.plot(kind='bar', colormap='viridis')
                    plt.title("Bar Chart of Spending")
                elif chart_type.value == 'Line':
                    data.plot(kind='line', marker='o', colormap='tab10')
                    plt.title("Line Chart of Spending")
                elif chart_type.value == 'Pie':
                    totals = data.sum()
                    if totals.sum() == 0:
                        print("No spending in this range/categories.")
                        return
                    totals.plot(kind='pie', autopct='%1.1f%%', startangle=140, shadow=True, colormap='tab20c')
                    plt.ylabel('')
                    plt.title("Pie Chart of Spending")
                elif chart_type.value == 'Stacked Bar':
                    data.plot(kind='bar', stacked=True, colormap='plasma')
                    plt.title("Stacked Bar Chart of Spending")
                elif chart_type.value == 'Cumulative Line':
                    data.cumsum().plot(kind='line', marker='o', colormap='cool')
                    plt.title("Cumulative Spending Over Time")
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print("Error plotting:", e)

    btn.on_click(plot_spending)
    display(widgets.VBox([cat_select, chart_type, date_start, date_end, btn, output]))


C:\Users\Scientist\AppData\Roaming\Python\Python313\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [6]:
def budget_analysis_widget():
    df = load_data()
    if df is None:
        print("No data available.")
        return
    budget_widgets = [widgets.FloatText(description=cat) for cat in fields[2:8]]
    btn = widgets.Button(description="Analyze Budget")
    output = widgets.Output()

    def analyze(_):
        with output:
            clear_output()
            budget_inputs = {fields[2+i]: budget_widgets[i].value for i in range(6)}
            total_spent = df[fields[2:8]].sum()
            for cat in total_spent.index:
                spent = total_spent[cat]
                print(f"{cat}: Spent={spent}, Budget={budget_inputs[cat]}, Diff={budget_inputs[cat]-spent}")
                if spent > budget_inputs[cat]:
                    print(f"Tip: Over budget on {cat}. Reduce spending!")
    btn.on_click(analyze)
    display(widgets.VBox(budget_widgets + [btn, output]))

budget_analysis_widget()


In [7]:
while True:
    print("\n--- Finance Tracker Menu ---")
    print("1. Enter Daily Spending")
    print("2. Enter Weekly Spending")
    print("3. Enter Monthly Spending")
    print("4. View Daily Spending")
    print("5. View Weekly Spending")
    print("6. View Monthly Spending")
    print("7. Exit")
    choice = input("Enter choice (1-7): ")
    if choice=="1": input_daily()
    elif choice=="2": input_weekly()
    elif choice=="3": input_monthly()
    elif choice=="4": view_spending("daily")
    elif choice=="5": view_spending("weekly")
    elif choice=="6": view_spending("monthly")
    elif choice=="7": break
    else: print("Invalid choice")



--- Finance Tracker Menu ---
1. Enter Daily Spending
2. Enter Weekly Spending
3. Enter Monthly Spending
4. View Daily Spending
5. View Weekly Spending
6. View Monthly Spending
7. Exit
Empty DataFrame
Columns: [Transport, Food, Data, School, Clothing, Others, Savings, Loans]
Index: []

--- Finance Tracker Menu ---
1. Enter Daily Spending
2. Enter Weekly Spending
3. Enter Monthly Spending
4. View Daily Spending
5. View Weekly Spending
6. View Monthly Spending
7. Exit

--- Finance Tracker Menu ---
1. Enter Daily Spending
2. Enter Weekly Spending
3. Enter Monthly Spending
4. View Daily Spending
5. View Weekly Spending
6. View Monthly Spending
7. Exit


ParserError: Error tokenizing data. C error: Expected 5 fields in line 7, saw 10
